In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W08`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W09`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W10`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W11`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
51668,2021-03-14 15:34:45+00:00,2021-03-14 15:35:12+00:00,Production,0047 of 0062,B2,None,None,KL13,Tue-NI Delivery,Sunday Afternoon,iXUsr_Wang(9965),4,0.45,2021-11
51667,2021-03-15 07:42:28+00:00,2021-03-15 07:44:40+00:00,Production,0038 of 0052,C4,None,None,KL5,Tue-NI Delivery,Monday Morning,iXUsr_Desry(7294),4,2.20,2021-11
58651,2021-03-15 11:36:52+00:00,2021-03-15 11:39:47+00:00,Production,0013 of 0024,C4,None,None,KL5,Wed-NI Delivery,Monday Morning,iXUsr_Chang(8908),5,2.92,2021-11
51666,2021-03-15 07:11:27+00:00,2021-03-15 07:15:10+00:00,Production,0032 of 0052,C4,None,None,KL5,Tue-NI Delivery,Monday Morning,iXUsr_Desry(7294),4,3.72,2021-11
14339,2021-03-12 08:59:51+00:00,2021-03-12 09:03:00+00:00,Production,None,C4,None,None,KL5,None,Friday Morning,iXUsr_Gael(7447),4,3.15,2021-11


In [5]:
data_in_use.shape

(68475, 14)